#  Exploring and Clustering of Neighborhoods in Toronto

### About the Project
> Each neighborhood in a city have unique offerings. But if we were moving to a new neighborhood, how could we know if a particular neighborhood is as per our preference or suitability? We can use Data Analysis method of CLustering to form clusters of the neighborhood as per inter-cluster similarities and intra-cluster dissimilarities. There are several clustering methodologies namely *KMeans Clustering, Hierarchical Clustering, DBSCAN Clustering* and many more. Here we will conduct a study on the neighborhoods of Toronto, Ontario and we will be using the **KMeans** method of clustering. Our approach will be to first gather the data for all the postal codes, boroughs and neighborhoods of TOronto. This we can do by performing webscraping activity from Wikipedia. Then we will map the entries with their corresponding latitude and longitude coordinates by fetching data from geocoder or from a predefined csv. Once this is done we will find the nearby venues to each of the nieghborhoods using Foursquare API. Then we prepare the data for performing the clustering analysis. Once we perform the KMeans cluster analysis we will examine the color coded location points on a Folium map and provide a comprehensive distinction between each of the clusters. 

For this exercise the first step would be to webscrape the postal codes and the corresponding boroughs and neighborhoods.

We will then load this webscaped data in a tabular form into a *pandas* dataframe.

### Installing the required libraries

We will use BeautifulSoup for webscraping

##### What is Beautiful Soup?
> *Beautiful Soup is a Python Library which is very useful for scraping data from webpages. It processes HTML and XML files and creates a parse tree by parsing the contents. This parsed data can be used by the programmer to extract contents from the webpages as per requirement.*

Installing the libraries for *BeautifulSoup* and *requests*

In [1]:
# we will use BeautifulSoup for webscraping
# installing the libraries for BeautifulSoup and requests

!pip install bs4
!pip install requests

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


### Importing the necessary libraries

In [2]:
from bs4 import BeautifulSoup 
import requests 
import pandas as pd
import numpy as np

### Webscraping

Webscraping from Wikipedia using **BeautifulSoup**.

The Wikipedia link is : <https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M>

Finding out how many tables present in the page

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data  = requests.get(url).text 
soup = BeautifulSoup(data,"html5lib")
#soup.prettify()
tables = soup.find_all('table')
print(len(tables))

3


### Data loading and cleaning

Finding out our required table containing the neighborhood data by using the *'Not assigned'* string

Using *find_all* to find the datapoints and loading them into *"neighborhood_data"*

transforming the scraped data to a pandas dataframe *"neighborhood_df"*

**Cleaning** the data

In [4]:
for index,table in enumerate(tables):
    if ("Not assigned" in str(table)):
        table_index = index
#print(table_index)

neighborhood_data = []

for row in tables[table_index].tbody.find_all("td"):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        neighborhood_data.append(cell)
        
#print(neighborhood_data)
neighborhood_df = pd.DataFrame(neighborhood_data)
#neighborhood_df

#print(neighborhood_df['Borough'].unique())
neighborhood_df['Borough']=neighborhood_df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
#print(neighborhood_df['Borough'].unique())
print("data cleaned.")

data cleaned.


In [5]:
# the dataframe

neighborhood_df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [6]:
# shape of dataframe

neighborhood_df.shape

(103, 3)

### Mapping latitude and longitude data with neighborhood dataframe

We will be using the *GeoSpatial Dataset*.

Loading the data from the csv and merging with the neighborhood dataframe.

In [7]:
csv_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv"
geo_data = pd.read_csv(csv_url)

geo_data = geo_data.rename(columns = {'Postal Code':'PostalCode'})

neighborhood_df_merged = pd.merge(neighborhood_df, geo_data, on='PostalCode')

neighborhood_df_merged
#geo_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


# Clustering

Now we are done with creating our dataset for performing the **Clustering** function.

The exploration part has been completed as we move on to the clustering part.

**How we will get our clusters?**

*In this clustering activity we will use the Foursquare API to find out the nearby venues for each of our neighborhood location. Then we will form 3 clusters for our neighborhood dataset based on their nearby venues and we will analyze each of the clusters formed.*

For the Clustering purpose we will only be considering the boroughs which have the word *'Toronto'* in them.

In [8]:
toronto_data = neighborhood_df_merged[neighborhood_df_merged['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
8,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259
9,M4J,East York/East Toronto,The Danforth East,43.685347,-79.338106


### Installing visualization and mapping libraries

We have the required dataset in toronto_data.

Now we will install all the required visualization and mapping libraries.

In [9]:
#!conda install -c conda-forge geopy --yes
!pip install folium
from geopy.geocoders import Nominatim

import matplotlib.cm as cm
import matplotlib.colors as colors

import folium

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


### Initial plotting

First let us plot all the datapoints on a map of Toronto prior to our Clustering activity.

In [10]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
#print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

![Map1](https://user-images.githubusercontent.com/83448048/121010188-b9c91000-c7b2-11eb-97b7-562b90850273.JPG)

### Defining the getNearbyVenues function

We define the getNearbyVenues function which takes the fields neighborhood name, latitude, longitude and search radius as its arguments and returns the nearby_venues dataframe.

In [11]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
       
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Getting the nearby venues

We define the Foursquare API credentials and use the getNearbyVenues fuction to get the nearby venues for the selected Toronto neighborhoods

In [12]:
CLIENT_ID = 'YYZUTUOAHZCE053T0MF24UC1N0Q4OTWXZ4XQH40ICCPBIIVJ' 
CLIENT_SECRET = 'J0A1PL1U4RVFVJOZHARQOSQPIKUBMAKSTHAL31SGLTEQZ24Q' 
VERSION = '20180605'
LIMIT = 100 

toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
First Canadi

### Grouping

Grouping the count of venues per neighborhood

In [13]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,47,47,47,47,47,47
"Brockton, Parkdale Village, Exhibition Place",21,21,21,21,21,21
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15
Central Bay Street,62,62,62,62,62,62
Christie,14,14,14,14,14,14
Church and Wellesley,66,66,66,66,66,66
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,27,27,27,27,27,27
Davisville North,10,10,10,10,10,10


### Onehot Encoding

Performing Onehot Encoding for each of the venue category per neighborhood

In [14]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 


fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.042553,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.000000,0.066667,0.066667,0.066667,0.066667,0.066667,0.133333,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,Central Bay Street,0.016129,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.016129,0.000000,0.000000,0.016129,0.000000
4,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
5,Church and Wellesley,0.015152,0.015152,0.015152,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.015152,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.015152
6,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000
7,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.037037,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
8,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Dufferin, Dovercourt Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000


### Finding the Most Common Venues

Here we find the most common venue per neighborhood

In [16]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))


neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Sandwich Place,Bakery,Farmers Market,Seafood Restaurant,Beer Bar,Vegetarian / Vegan Restaurant,French Restaurant,Diner
1,"Brockton, Parkdale Village, Exhibition Place",Sandwich Place,Coffee Shop,Breakfast Spot,Café,Convenience Store,Restaurant,Japanese Restaurant,Italian Restaurant,Intersection,Bar
2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Terminal,Coffee Shop,Sculpture Garden,Boutique,Rental Car Location,Bar,Plane,Harbor / Marina,Boat or Ferry,Airport Service
3,Central Bay Street,Coffee Shop,Sandwich Place,Sushi Restaurant,Italian Restaurant,Japanese Restaurant,Café,Restaurant,Salad Place,Bank,Burger Joint
4,Christie,Grocery Store,Café,Park,Baby Store,Italian Restaurant,Restaurant,Coffee Shop,Nightclub,Discount Store,Diner


### KMeans Clustering

We perform the *KMeans Clustering* and form the final dataframe for plotting the clusters

In [17]:
from sklearn.cluster import KMeans

kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

#kmeans.labels_[0:10] 

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.shape

(39, 16)

### Plotting the Clusters

We plot the 3 clusters on the map of Toronto

In [18]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0.5, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

![Map2](https://user-images.githubusercontent.com/83448048/121010490-1a584d00-c7b3-11eb-8f08-32a2f176cac6.JPG)

### Cluster 1

Neighborhoods having Parks and Trails as the most common nearby venues. These neighborhoods will be suitable for people who would like to go for frequent walks or for families with kids who are likely to frequent the parks

In [19]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,The Danforth East,0,Park,Intersection,Convenience Store,Cuban Restaurant,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop
18,Lawrence Park,0,Park,Business Service,Bus Line,Swim School,College Arts Building,College Gym,Dog Run,Distribution Center,Discount Store,Diner
21,Forest Hill North & West,0,Park,Trail,Jewelry Store,Bus Line,Sushi Restaurant,College Arts Building,Cuban Restaurant,Dog Run,Distribution Center,Clothing Store
33,Rosedale,0,Park,Trail,Playground,Convenience Store,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega


### Cluster 2

The neighborhood in this cluster seems more residential as the nearby venues show restaurants and dog run among the most common places.

In [20]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,"Moore Park, Summerhill East",1,Restaurant,Wine Shop,Creperie,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega


### Cluster 3

The neighborhoods in this cluster shows high concentraion of Coffee Shops, Cafes, Pubs, Bars, Fast food joints, etc, ie. locations for hanging out or socializing. Hence these neighborhoods are more likely to be commercial areas having less residentials

In [21]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",2,Coffee Shop,Bakery,Park,Pub,Café,Greek Restaurant,Event Space,Bank,Farmers Market,Spa
1,"Garden District, Ryerson",2,Coffee Shop,Clothing Store,Café,Hotel,Middle Eastern Restaurant,Pizza Place,Cosmetics Shop,Japanese Restaurant,Sandwich Place,Falafel Restaurant
2,St. James Town,2,Coffee Shop,Cocktail Bar,Café,Italian Restaurant,Restaurant,Clothing Store,Cosmetics Shop,Gym,Gastropub,Theater
3,The Beaches,2,Health Food Store,Playground,Pub,Wine Shop,Cosmetics Shop,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store
4,Berczy Park,2,Coffee Shop,Cocktail Bar,Sandwich Place,Bakery,Farmers Market,Seafood Restaurant,Beer Bar,Vegetarian / Vegan Restaurant,French Restaurant,Diner
5,Central Bay Street,2,Coffee Shop,Sandwich Place,Sushi Restaurant,Italian Restaurant,Japanese Restaurant,Café,Restaurant,Salad Place,Bank,Burger Joint
6,Christie,2,Grocery Store,Café,Park,Baby Store,Italian Restaurant,Restaurant,Coffee Shop,Nightclub,Discount Store,Diner
7,"Richmond, Adelaide, King",2,Coffee Shop,Café,Sandwich Place,Clothing Store,Gym,Restaurant,Sushi Restaurant,Deli / Bodega,Bank,Hotel
8,"Dufferin, Dovercourt Village",2,Supermarket,Bar,Brewery,Bakery,Bank,Café,Middle Eastern Restaurant,Pharmacy,Liquor Store,Grocery Store
10,"Harbourfront East, Union Station, Toronto Islands",2,Coffee Shop,Hotel,Café,Scenic Lookout,Pizza Place,Aquarium,Gym,Sandwich Place,Bank,Boat or Ferry


## Summary

**Hence after performing our *Exploring and Clustering Neighborhoods in Toronto* activity, we have a more comprehensive idea about the various neighborhoods in the City of Toronto.**